# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [1]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [3]:
# Dataset name
dataset_name = "facebook/ExploreToM"

# TODO: load and explore the dataset
dataset = load_dataset(dataset_name)

In [10]:
# TODO: number of rows in the dataset
print(dataset.shape)

# TODO: Keys in the dataset
print(dataset.keys())

# TODO: Feature names
print(dataset['train'].features)

# TODO: Display a single row
idx = 10000
for k, v in dataset['train'][idx].items():
   print(f'key: {k}')
   print(v)
   print()


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value(dtype='string', id=None), 'infilled_story': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'expected_answer': Value(dtype='string', id=None), 'qprop=params': Value(dtype='string', id=None), 'qprop=nth_order': Value(dtype='int64', id=None), 'qprop=non_unique_mental_state': Value(dtype='bool', id=None), 'sprop=is_false_belief_story_1st': Value(dtype='bool', id=None), 'sprop=is_false_belief_story_1st_and_2nd': Value(dtype='bool', id=None), 'sprop=story_accuracy_1st_raw': Value(dtype='float64', id=None), 'sprop=story_accuracy_1st_infilled': Value(dtype='float64', id=None), 'sprop=global_idx': Value(dtype='int64', id=None), 'param=story_type': Value(dtype='string', id=None), 'param=num_stories_total': Value(dtype='int64', id=None), 'param=max_sentences': Value(dtype='int64', id=None), 'param=num_people': Value(dtype='int64', id=None), 'param=num_moves': Value(dtype='int64', id=None), 'param=nu

In [15]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [18]:
# TODO: Summarise the text with the pipeline's default model
idx = 100 
summarise = pipeline('summarization')
text = dataset['train'][idx]['story_structure']
result = summarise(text, min_length=5, max_length=30)
print(text)
print('-------------------')
print(result)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Leslie entered the main tent. Leslie left the main tent. Isabella entered the storage trailer. Isabella moved the stuffed rabbit to the wooden chest, which is also located in the storage trailer. Leslie entered the main tent. Isabella moved the stuffed rabbit to the main tent, leaving the wooden chest in its original location. Isabella told out loud about the festival marketing strategies. Isabella told privately to Colton that Leslie is in the main tent. While this action was happening, Leslie witnessed this action in secret (and only this action).
-------------------
[{'summary_text': ' Leslie entered the main tent. Isabella moved the stuffed rabbit to the wooden chest, which is also located in the storage trailer . Leslie'}]


In [20]:
qna = pipeline('question-answering')

idx = 5
text = dataset['train'][idx]['story_structure']
question = dataset['train'][idx]['question']

result = qna(context=text, question=question)

print(result)

print(dataset['train'][idx]['expected_answer'])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'score': 0.7939401865005493, 'start': 367, 'end': 384, 'answer': 'leather briefcase'}
leather briefcase


## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [21]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [22]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [23]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [24]:
# TODO: Encode text
message = 'big black bug bleeds black blood'

encoded = tokenizer(message, return_tensors='pt')

print(encoded)

{'input_ids': tensor([[  101,  1992,  1602, 15430, 24752,  1116,  1602,  1892,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [25]:
# TODO: Encoding multiple texts
texts = [
   message,
   "row row row your boat, gently down the stream"
]

encoded = tokenizer(texts, return_tensors='pt', padding=True)

print(encoded)


{'input_ids': tensor([[  101,  1992,  1602, 15430, 24752,  1116,  1602,  1892,   102,     0,
             0,     0],
        [  101,  5105,  5105,  5105,  1240,  3499,   117,  4588,  1205,  1103,
          5118,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [28]:
# TODO: Decode text

decoded = tokenizer.decode(encoded['input_ids'][1], skip_special_tokens=True)

print(decoded)


row row row your boat, gently down the stream


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [29]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering 
import torch

In [55]:
# TODO: Load question answer model
idx = 20 
context = dataset['train'][idx]['story_structure']
context = dataset['train'][idx]['infilled_story']
question = dataset['train'][idx]['question']
expected_answer = dataset['train'][idx]['expected_answer']



In [56]:
# TODO: Encode context and question

enc_context = tokenizer(context, return_tensors='pt')
enc_question = tokenizer(question, return_tensors='pt')

inputs = tokenizer(context, question, return_tensors='pt', padding=True)
print(inputs)


{'input_ids': tensor([[  101,  1109,  3336,  1125,  3742, 15495,   117,  9616,   170,  3258,
          8656,  1166,  1103, 16118,  1979,  5184,   119,  1130,  1103,  1707,
          1395,   117,   170,  5495,  1104,  5444,  5097,   117,  8113,   188,
         21200,  1279,   117,  1105,  4301, 18217,  1116,  7648,  1103,  4122,
          7072,   117,  4405,  1118,  1103,  7859,  5974,  1104,  1486, 27650,
          1105,  4489,  6628,   119,  1109,  1442,   172, 28051,  1501,   117,
          1105, 23929,  8495,  1389,  2045,  1107,   117,  1117,  1257, 15049,
          1103, 22911,  1707,  1395,   117,  1229,  4899,  1224,   117, 20587,
          1145,  2242,  1103,  2000,   117,  1123,  2806, 16708,  1166,  1103,
          1395,   112,   188,  9726,   119,  1249, 20587,  3378,  1113, 10883,
         16504,  1283,  1103,  8113,  6665,   117,  1123,  2209,  4003,  4359,
          1111,   170,  3325,  1248,   117, 23438,  2368, 23929,  8495,  1389,
           170,  4142,  2487,  1106,  

In [41]:
# TODO: Tokenize the inputs
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


In [60]:
result = model(inputs['input_ids'], inputs['attention_mask'])



In [32]:
# Ensure minimum and maximum token length in the answer
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [61]:
#print(result)
start_text = torch.argmax(result.start_logits)
end_text = torch.argmax(result.end_logits)
answer_ids = inputs['input_ids'][0][start_text: end_text + 1 ]
print(answer_ids)

tensor([101])


In [62]:
decoded_ans = tokenizer.decode(answer_ids, skip_special_tokens=True)
print(decoded_ans, expected_answer)

 cardboard box


In [ ]:
# TODO Return a minimum of 5 tokens


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many stories has Dickens wrote?"

